In [79]:
import pandas as pd
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
sw=stopwords.words('english')
ps=PorterStemmer()

In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT 
CONVERT(NVARCHAR(15) , IdText) as IDT , 
REPLACE(IdText, '.','_') as IDT_ , 
CASE 
WHEN CONVERT(NVARCHAR(9),Section_Count) = '0' THEN '1a'
WHEN CONVERT(NVARCHAR(9),Section_Count) = '1' THEN '1b'
ELSE  CONVERT(NVARCHAR(9),Section_Count) 
END AS SEC 
, CONVERT(NVARCHAR(9) , QPK) as QPK
, QTEXT as QTXT 
FROM vw_MetricsCompositeKeys WHERE PK_FORM = '2021-A-HVA'
ORDER BY Section_Count ASC, SortPos ASC
""" 
df = pd.read_sql(query,con=conn)   
df.to_excel(r'C:\Users\Tim\Documents\1802\BOD1802_2021.xlsx','AllMetrics') 
df

In [ ]:
df=df.loc[ df.IDT_.isin(['4_1_1','6_2_1','6_2_2','6_2_3','6_2_4','6_2_4_1','6_3_1_1' ]) ]
df

In [ ]:
aspx = '' 
for i,r in df.iterrows():  
    s = open(f'C:\\_som\\_src\\TXT.aspx', 'r').read()
    s = s.replace('{0}',str(r.QPK)).replace('{1}',str(r.IDT_)).replace('{4}',str(r.IDT_)+ ' : ' +str(r.QPK)+ '\n\t' +str(r.QTXT)).replace('{5}','')
    s = s.replace('{2}',str(160))
    df.loc[i, 'ASPX'] = s
    aspx = aspx + s + '\n'
    print(df.loc[i, 'ASPX'] + '\n\n') 
  


In [ ]:
with open(r'C:\\_som\\_cache.txt', 'w') as file:
  file.write(aspx)

In [60]:
conn.close()

In [93]:
t = """
Languages technologies platforms is your web don't we have web driver 
"""
def encode(t):
    t = ' '.join([w for w in t.split(' ') if w not in sw])
    t = ' '.join([ps.stem(w) for w in t.split(' ')])
    t = t.upper()
    t = re.sub('[^A-Z0-9]', '', t )   
    t = re.sub('[AEIOUY]', '', t)  
    return t
encode(t)

'LNGGTCHNLGPLTFRMWBWBDRVR'